In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix
)



In [2]:
# Load Monthly CRSP

CRSP_PATH = '../data/monthly_crsp.csv'
df_crsp = pd.read_csv(
    CRSP_PATH ,
    parse_dates=['MthCalDt'],
    usecols=['PERMNO','CUSIP','MthCalDt','MthRet']
)


# Load Compustat Fundamentals

COMP_PATH = '../data/CompFirmCharac.csv'

df_comp = pd.read_csv(
    COMP_PATH,
    parse_dates=['datadate'], dayfirst=True,
)


C:\Users\sinna\AppData\Local\Temp\ipykernel_6136\616781079.py:15: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comp = pd.read_csv(
C:\Users\sinna\AppData\Local\Temp\ipykernel_6136\616781079.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_comp = pd.read_csv(


### CLEAN CRSP


In [3]:
# 2.1: Keep only rows where MthRet is available and cast to float
df_crsp = df_crsp.dropna(subset=['MthRet']).copy()
df_crsp['MthRet'] = df_crsp['MthRet'].astype(float)

# 2.2: Sort by CUSIP, date so that shift is correct
df_crsp['date'] = pd.to_datetime(df_crsp['MthCalDt'].astype(str), format='mixed')
df_crsp = df_crsp.sort_values(['CUSIP','date']).reset_index(drop=True)

# 2.3: Create next‐month return target (binary)
# df_crsp['Ret_t1'] = df_crsp.groupby('CUSIP')['MthRet'].shift(-1)
df_crsp['y'] = df_crsp.groupby('CUSIP')['MthRet'].shift(-1)
# df_crsp['y'] = (df_crsp['Ret_t1'] > 0).astype(int)
df_crsp = df_crsp.dropna(subset=['y']).copy()

In [4]:
import calendar

# Add technical indicators and months

def compute_close(y_series):
    close = (1 + y_series.fillna(0)).cumprod()
    close.iloc[0] = 1.0
    return close

df_crsp = df_crsp.sort_values(["PERMNO", "date"])
df_crsp["close"] = df_crsp.groupby("PERMNO")["MthRet"].apply(compute_close).reset_index(level=0, drop=True)

def calculate_rsi(series, window=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.rolling(window).mean()
    ma_down = down.rolling(window).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

# Group-wise calculations
def add_technical_indicators(group):
    group = group.copy()
    group['EMA_Close'] = group['close'].ewm(span=14, adjust=False).mean()
    group['EMA'] = (group['close'] - group['EMA_Close'])/ group['close']

    rolling_std = group['close'].rolling(window=14).std()
    group['Volatility'] = rolling_std / group['close']

    group['RSI'] = calculate_rsi(group['close'])
    
    # MACD
    ema12 = group['close'].ewm(span=12, adjust=False).mean()
    ema26 = group['close'].ewm(span=26, adjust=False).mean()
    group['MACD_diff'] = ema12 - ema26
    group['MACD_Signal'] = group['MACD_diff'].ewm(span=9, adjust=False).mean()
    group['MACD'] = (group['MACD_diff'] - group['MACD_Signal']) / group['close']


    return group

# Apply technical indicators to each stock (PERMNO)
df_crsp = df_crsp.groupby("PERMNO", group_keys=False).apply(add_technical_indicators, include_groups=False)

df_crsp.drop(columns=["close", "EMA_Close", "MACD_diff", "MACD_Signal", 'y_forward'], inplace=True, errors='ignore')

# Add months
df_crsp['month'] = df_crsp['date'].dt.month.map(lambda x: calendar.month_name[x])

# Create dummy variables with month names as column names
month_dummies = pd.get_dummies(df_crsp['month']).astype(int)

# Concatenate dummies with original dataframe
df_crsp = pd.concat([df_crsp, month_dummies], axis=1)

# drop the intermediate 'month' column 
df_crsp = df_crsp.drop(columns=['month'])

# Get the current columns
cols = list(df_crsp.columns)

# Move 'y' to the end if it exists
if 'y' in cols:
    cols.remove('y')
    cols.append('y')

# Reorder the DataFrame columns
df_crsp = df_crsp[cols]

df_crsp = df_crsp.dropna()
print(df_crsp)

            CUSIP   MthCalDt    MthRet       date       EMA  Volatility  \
2970518  68391610 1987-03-31 -0.384615 1987-03-31 -4.416586    6.008675   
2970519  68391610 1987-04-30 -0.062500 1987-04-30 -4.140666    6.416235   
1746075  39040610 1987-03-31  0.037486 1987-03-31  0.028843    0.058017   
1746076  39040610 1987-04-30 -0.039216 1987-04-30 -0.009357    0.057093   
1746077  39040610 1987-05-29 -0.071429 1987-05-29 -0.075400    0.065004   
...           ...        ...       ...        ...       ...         ...   
3917811  88160R10 2024-07-31  0.172781 2024-07-31  0.094132    0.146526   
3917812  88160R10 2024-08-30 -0.077390 2024-08-30  0.015727    0.148719   
3917813  88160R10 2024-09-30  0.221942 2024-09-30  0.168567    0.119113   
3917814  88160R10 2024-10-31 -0.045025 2024-10-31  0.112118    0.121613   
3917815  88160R10 2024-11-29  0.381469 2024-11-29  0.309653    0.131800   

               RSI      MACD  April  August  ...  February  January  July  \
2970518  31.218276 -1.

In [5]:
# 2.4: Generate price‐history features (momentum + volatility)
#
#   - 3M momentum: cumulative return over past 3 months (t-3 → t-1)
#   - 6M momentum: cumulative return over past 6 months
#   - 12M momentum: cumulative return over past 12 months
#   - 12M rolling volatility: std of monthly returns over past 12 months
#
def compute_momentum_and_vol(df):
    df = df.sort_values('date')
    # Rolling log(1+return), because cumulative product of (1 + ret) = exp(sum(log(1+ret)))
    df['log1p_ret'] = np.log1p(df['MthRet'])
    df['log1p_ret_shift1'] = df.groupby('CUSIP')['log1p_ret'].shift(1)
    df['cum12_1_log'] = df.groupby('CUSIP')['log1p_ret_shift1'].rolling(window=11).sum().reset_index(0,drop=True)
    df['mom_12_1'] = np.expm1(df['cum12_1_log'])
    df['cum3m_log'] = df.groupby('CUSIP')['log1p_ret'].rolling(window=3, min_periods=3).sum().reset_index(0,drop=True)
    df['cum6m_log'] = df.groupby('CUSIP')['log1p_ret'].rolling(window=6, min_periods=6).sum().reset_index(0,drop=True)
    df['cum12m_log'] = df.groupby('CUSIP')['log1p_ret'].rolling(window=12, min_periods=12).sum().reset_index(0,drop=True)
    df['momentum_3m'] = np.expm1(df['cum3m_log'])    # exp(sum)-1 => (1+r1)*(1+r2)*(1+r3) - 1
    df['momentum_6m'] = np.expm1(df['cum6m_log'])
    df['momentum_12m'] = np.expm1(df['cum12m_log'])
    df['volatility_12m'] = df.groupby('CUSIP')['MthRet'].rolling(window=12, min_periods=12).std().reset_index(0,drop=True)
    # Drop intermediate log columns
    return df.drop(columns=['log1p_ret','cum3m_log','cum6m_log','cum12m_log'])

df_crsp = compute_momentum_and_vol(df_crsp)

# 2.5: Trim CUSIP to 8 characters (for merging) and drop NA
df_crsp['cusip'] = df_crsp['CUSIP'].astype(str).str[:8]
df_crsp = df_crsp.dropna(subset=['cusip']).copy()

### CLEAN COMPFIRM

In [6]:
# 3.1: Keep only Industrial & Consolidated
df_comp = df_comp[
    (df_comp['consol'] == 'C')
].copy()

# 3.2: Trim & parse keys/dates
df_comp['cusip'] = df_comp['cusip'].astype(str).str[:8]
df_comp['datadate'] = pd.to_datetime(df_comp['datadate'])
df_comp = df_comp.dropna(subset=['cusip','datadate']).copy()

# 3.3: Build “effective_date” = datadate + 45 calendar days,
#      so that we only use Q data ~45 days after quarter‐end.
df_comp['effective_date'] = df_comp['datadate'] + pd.Timedelta(days=45)
df_comp = df_comp.set_index('effective_date').sort_index()

# 3.4: Select a larger fundamental set (YTD flows + per‐share metrics)
fundamental_cols = [
    'revty',    # Revenue YTD
    'saley',    # Sales YTD
    'capxy',    # CapEx YTD
    'oibdpy',   # EBITDA YTD
    'rdipay',   # R&D expense YTD
    'xsgay',    # SG&A expense YTD
    'txpdy',    # Tax provision YTD
    'epsfxy',   # Diluted EPS ex‐extra YTD
    'cshfdy',   # Diluted shares YTD (millions)
    'xoptepsy'  # Option expense per share YTD
]

df_comp_small = df_comp[['cusip'] + fundamental_cols].copy()

# 3.5: For each “cusip + quarter,” drop exact duplicates
df_comp_small = df_comp_small.reset_index().drop_duplicates(
    subset=['cusip','effective_date']
).set_index('effective_date').sort_index()

### MERGE

In [7]:
# 4.1: Set df_crsp index to “date”
df_crsp = df_crsp.set_index('date').sort_index()

# 4.2: Merge (for every month, get the most recent Compustat row ≤ that month’s date)
df_merged = pd.merge_asof(
    left = df_crsp.reset_index(),
    right = df_comp_small.reset_index(),
    left_on = 'date',
    right_on = 'effective_date',
    by = 'cusip',
    direction = 'backward',
    allow_exact_matches=True
).set_index('date')

# 4.3: Drop rows where any of our fundamentals are NA, since we can’t compute ratios otherwise
df_merged = df_merged.dropna(subset=fundamental_cols + ['y']).copy()


In [8]:
df = df_merged.copy()

# 5.1: Engineer simple ratios (safe‐guard divisions by zero)
df['EBIT_margin']      = df['oibdpy'] / df['saley'].replace({0: np.nan})
df['R&D_intensity']    = df['rdipay'] / df['saley'].replace({0: np.nan})
df['SGA_intensity']    = df['xsgay'] / df['saley'].replace({0: np.nan})
df['Tax_rate']         = df['txpdy']  / df['oibdpy'].replace({0: np.nan})
df['Capex_to_Revenue'] = df['capxy']  / df['revty'].replace({0: np.nan})

# 5.2: Compute QoQ growth rates on YTD fundamentals
for col in ['revty','oibdpy','rdipay','xsgay']:
    df[col + '_QoQ_growth'] = df.groupby('cusip')[col].pct_change()  # (This QY / Last QY) - 1

# 5.3: Optionally drop some raw dollar‐amount YTD columns if you want just ratios
#      (otherwise let the model pick scale vs. ratio)
# df = df.drop(columns=['revty','saley','capxy','oibdpy','rdipay','xsgay','txpdy','epsfxy','cshfdy','xoptepsy'])

### TRAINING

In [9]:
feature_columns = [
    # Price/technical:
    'momentum_3m', 'momentum_6m', 'momentum_12m', 'volatility_12m',

    # Basic YTD fundamentals (optional—tree can split on scale):
    'revty', 'saley', 'capxy', 'oibdpy', 'rdipay', 'xsgay', 'txpdy', 'epsfxy', 'cshfdy', 'xoptepsy',

    # Engineered ratios:
    'EBIT_margin', 'R&D_intensity', 'SGA_intensity', 'Tax_rate', 'Capex_to_Revenue',

    # QoQ growth rates:
    'revty_QoQ_growth', 'oibdpy_QoQ_growth', 'rdipay_QoQ_growth', 'xsgay_QoQ_growth'
]

# Drop any rows where engineered features are NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=feature_columns + ['y']).copy()


X = df[feature_columns]
y = df['y']

In [10]:
# Instead of a fixed 80/20 cutoff, build an expanding‐window cross‐validation
# but keep a final out‐of‐sample test set (last 20% of months).
n_obs = len(df)
cutpoint = int(n_obs * 0.8)

X_train = X.iloc[:cutpoint]
y_train = y.iloc[:cutpoint]

X_test  = X.iloc[cutpoint:]
y_test  = y.iloc[cutpoint:]

In [13]:

# pipe = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler',   StandardScaler()),   # scale ratio features so splits are easier
#     ('clf',      RandomForestClassifier(random_state=42, class_weight='balanced'))
# ])

# param_grid = {
#     'clf__n_estimators': [100, 200],
#     'clf__max_depth': [5, 7, 9],
#     'clf__max_features': ['sqrt', 'log2']
# }

from sklearn.ensemble import RandomForestRegressor

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('reg', RandomForestRegressor(random_state=42))
])

param_grid = {
    'reg__n_estimators': [100, 200],
    'reg__max_depth': [5, 7, 9],
    'reg__max_features': ['sqrt', 'log2']
}


tscv = TimeSeriesSplit(n_splits=5)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=tscv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_


Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\sinna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters: {'reg__max_depth': 5, 'reg__max_features': 'sqrt', 'reg__n_estimators': 100}


In [15]:
# # 9.1: Prediction & Classification Report
# y_pred = best_model.predict(X_test)
# print("\nClassification Report on Test Set:")
# print(classification_report(y_test, y_pred))

# # 9.2: ROC AUC + Precision‐Recall AUC
# y_proba = best_model.predict_proba(X_test)[:, 1]
# roc_auc = roc_auc_score(y_test, y_proba)

# precision, recall, _ = precision_recall_curve(y_test, y_proba)
# pr_auc = auc(recall, precision)

# print(f"Test ROC AUC:  {roc_auc:.4f}")
# print(f"Test PR AUC:   {pr_auc:.4f}")

# # 9.3: Display a confusion matrix if you like
# cm = confusion_matrix(y_test, y_pred)
# print("Confusion Matrix (low values = better balance):\n", cm)

# # ───────────
# # 10. FEATURE IMPORTANCE AND NEXT STEPS
# # ───────────

# importances = best_model.named_steps['clf'].feature_importances_
# feat_imp = pd.Series(importances, index=feature_columns).sort_values(ascending=False)
# print("\nTop 10 Feature Importances:")
# print(feat_imp.head(10))

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model.predict(X_test)

print("\nRegression Metrics on Test Set:")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
print(f"R²:   {r2_score(y_test, y_pred):.4f}")



Regression Metrics on Test Set:
MAE:  0.0769
MSE:  0.0125
R²:   0.0057


TRAINING 2

In [16]:
feature_columns = [
    # Price/technical:
    'momentum_3m', 'momentum_6m', 'mom_12_1', 'volatility_12m',

    # Basic YTD fundamentals (optional—tree can split on scale):
    'revty', 'saley', 'capxy', 'oibdpy', 'rdipay', 'xsgay', 'txpdy', 'epsfxy', 'cshfdy', 'xoptepsy',

    # Engineered ratios:
    'EBIT_margin', 'R&D_intensity', 'SGA_intensity', 'Tax_rate', 'Capex_to_Revenue',

    # QoQ growth rates:
    'revty_QoQ_growth', 'oibdpy_QoQ_growth', 'rdipay_QoQ_growth', 'xsgay_QoQ_growth'
]

# Drop any rows where engineered features are NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=feature_columns + ['y']).copy()


X = df[feature_columns]
y = df['y']

In [17]:
# Instead of a fixed 80/20 cutoff, build an expanding‐window cross‐validation
# but keep a final out‐of‐sample test set (last 20% of months).
n_obs = len(df)
cutpoint = int(n_obs * 0.8)

X_train = X.iloc[:cutpoint]
y_train = y.iloc[:cutpoint]

X_test  = X.iloc[cutpoint:]
y_test  = y.iloc[cutpoint:]

In [19]:
from xgboost import XGBClassifier

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',   StandardScaler()),  
    ('clf',      XGBClassifier(
         objective='binary:logistic',
         eval_metric='auc',
         use_label_encoder=False,
         n_estimators=200,
         max_depth=5,
         learning_rate=0.05,
         random_state=42
    ))
])

param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [3, 5, 7],
    'clf__learning_rate': [0.01, 0.05]
}

grid = GridSearchCV(pipe, param_grid, cv=TimeSeriesSplit(n_splits=5), scoring='roc_auc', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
print("XGBoost Best params:", grid.best_params_)
best_model = grid.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sinna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sinna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sinna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8

In [ ]:
# 9.1: Prediction & Classification Report
y_pred = best_model.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

# 9.2: ROC AUC + Precision‐Recall AUC
y_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_proba)

precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)

print(f"Test ROC AUC:  {roc_auc:.4f}")
print(f"Test PR AUC:   {pr_auc:.4f}")

# 9.3: Display a confusion matrix if you like
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix (low values = better balance):\n", cm)

# ───────────
# 10. FEATURE IMPORTANCE AND NEXT STEPS
# ───────────

importances = best_model.named_steps['clf'].feature_importances_
feat_imp = pd.Series(importances, index=feature_columns).sort_values(ascending=False)
print("\nTop 10 Feature Importances:")
print(feat_imp.head(10))


Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.44      0.37      0.40       355
           1       0.63      0.70      0.66       541

    accuracy                           0.57       896
   macro avg       0.53      0.53      0.53       896
weighted avg       0.55      0.57      0.56       896

Test ROC AUC:  0.5590
Test PR AUC:   0.6516
Confusion Matrix (low values = better balance):
 [[131 224]
 [165 376]]

Top 10 Feature Importances:
oibdpy_QoQ_growth    0.053566
xsgay_QoQ_growth     0.051362
xoptepsy             0.050947
oibdpy               0.050897
Tax_rate             0.050676
Capex_to_Revenue     0.049257
cshfdy               0.049228
mom_12_1             0.048768
xsgay                0.048410
rdipay               0.046477
dtype: float32
